# Importaciones

In [1]:

from datetime import datetime
from datetime import timedelta
import concurrent.futures
from pytz import timezone

import psycopg
import newspaper

from extract_news import news_extractor_per_media, news_extractor_per_media_concurrent
from medias.medias_list import medias_dict

# Notas

* Documentación de [psycopg3](https://www.psycopg.org/psycopg3/docs/basic/index.html)
* Documentación de [Newspaper4k](https://pypi.org/project/newspaper4k/)
* *connections string*: https://www.postgresql.org/docs/current/libpq-connect.html#LIBPQ-CONNSTRING

Error al usar la función `build` de *newspaper*:

* [AttributeError: 'NoneType' object has no attribute 'xpath'](https://github.com/codelucas/newspaper/issues/646)

# Base de datos

In [2]:
conn_postgresql = psycopg.connect(
    host="localhost",
    dbname="news_kg_v1",
    user="postgres",
    password="0rioN-689"
)

In [ ]:
conn_postgresql.execute(
'''
CREATE TABLE IF NOT EXISTS news_chile (
  id SERIAL PRIMARY KEY,
  media_name VARCHAR(150) NULL,
  url TEXT NULL,
  date TIMESTAMP NULL,
  author VARCHAR(150) NULL,
  body TEXT NULL,
  keywords TEXT NULL
);
'''
)

In [ ]:
conn_postgresql.commit()

In [ ]:
conn_postgresql.close()

## Populando base de datos

In [3]:
today = datetime.today()
tz = timezone('UTC')
today = today.replace(tzinfo=tz)
time_delta = timedelta(days=1, hours=today.hour, minutes=today.minute)

In [4]:
news_extractor_per_media(
    country='chile',
    medias=medias_dict,
    today=today,
    t_delta=time_delta,
    conn=conn_postgresql
)

c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages\newspaper\source.py:260: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if feed.doc:


Ha ocurrido un error con https://www.aricamia.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://elreporterodeiquique.com/:
Article is binary data: https://elreporterodeiquique.com/category/deportes/feed
Ha ocurrido un error con https://ovallehoy.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://www.observador.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://www.elinformador.cl/:
'NoneType' object has no attribute 'xpath'


c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Ha ocurrido un error con https://www.laizquierdadiario.cl/Chile:
Http error when downloading https://www.laizquierdadiario.cl/Chile. Status code: {status_code}
Ha ocurrido un error con https://www.biobiochile.cl/:
Article is binary data: https://www.biobiochile.cl/frecuencias_rbb.pdf
Ha ocurrido un error con https://www.elperiscopio.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://www.theclinic.cl/:
Http error when downloading https://www.theclinic.cl/. Status code: {status_code}


In [ ]:
with concurrent.futures.ThreadPoolExecutor() as tpe:
    results = [tpe.submit(news_extractor_per_media_concurrent, country='chile', media=media, today=today, t_delta=time_delta, conn=conn_postgresql) for media in medias_dict['chile']]